# Machine Learning Component

<span style="color:red">TODO:</span> Ask David how he dealt with imaginary numbers in mean spectral energy

1 -> Import Data </n>
2 -> Calculate Features </n>
* Mean
* Variance
* Standard Deviation
* Maximum
* Minimum
* Skewness
* Kurtosis
* Mean crosings
* Mean spectral Energy
* 4 Bin Histogram (4 features)

3-> Create classifier


4 -> Make Testing feature vector

5 -> Test classifier

6 -> Create confusion matrix

Note: This script works if you run all cells in order.

## Calculate Features

In [117]:
import math
import scipy as sp
from scipy.stats import kurtosis, skew
import sklearn
import numpy as np
import pandas as pd

In [118]:
data = pd.read_csv('Elbowing.txt', sep="\t", header=None)
#data.columns = ["R_BI", "R_TRI", "L_BI", "L_TRI", "R_THI", "R_HAM", "L_THI", "L_HAM"]
elbow = data[0:700]

data = pd.read_csv('Frontkicking.txt', sep="\t", header=None)
frontKick = data[0:700]

data = pd.read_csv('Hamering.txt', sep="\t", header=None)
hamering = data[0:700]

data = pd.read_csv('Headering.txt', sep="\t", header=None)
headering = data[0:700]

frames = [elbow, frontKick, hamering, headering]
#combined = pd.concat(frames)
#np.shape(combined)

In [119]:
def appendEnergyVector(dataFrame):
    #take in the dataFrame, set new columns as a^2 + b^2 for each pair of 2. 
    energyCols = [8, 9, 10, 11]
    i = 0
    for col in energyCols:
        dataFrame[col] = np.square(dataFrame[i])+np.square(dataFrame[i+1])
        i = i+2
    return dataFrame

elbow = appendEnergyVector(elbow)
frontKick = appendEnergyVector(frontKick)
hamering = appendEnergyVector(hamering)
headering = appendEnergyVector(headering)

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [120]:
#Mean crossings
def meanCrossing(frame):
    centeredFrame = frame - np.mean(frame)
    return ((centeredFrame[0][:-1].values * centeredFrame[0][1:].values) < 0).sum()

In [121]:
#4 bin histogram: It's actually 4 features. The bins are range/4. Check frequency of each bin for each feature. 
def fourBin(frame):
    allBins = np.histogram(frame[0],bins=4)[0][:]
    for i in range(1,12):
        allBins = np.append(allBins, np.histogram(frame[i],bins=4)[0][:])
    return allBins

In [122]:
featMean = np.empty([4,12])
featVar = np.empty([4,12])
featStd = np.empty([4,12])
featMin = np.empty([4,12])
featMax = np.empty([4,12])
featSkew = np.empty([4,12])
featKurtosis = np.empty([4,12])
featMeanCrossing = np.empty([4,12])
featMeanSpectralEnergy = np.empty([4,12], dtype=complex)
featFourBin = np.empty([4,48])

i=0
for frame in frames:
    featMean[i][:] = np.mean(frame).values.reshape([1,12])
    featVar[i][:] = np.var(frame).values.reshape([1,12])
    featStd[i][:] = np.std(frame).values.reshape([1,12])
    featMin[i][:] = np.min(frame).values.reshape([1,12])
    featMax[i][:] = np.max(frame).values.reshape([1,12])
    featSkew[i][:] = skew(frame)
    featKurtosis[i][:] = kurtosis(frame)
    featMeanCrossing[i][:] = meanCrossing(frame)
    featMeanSpectralEnergy[i][:] = np.mean(np.square(np.fft.fft(frame)),0)
    featFourBin[i][:] = fourBin(frame)
    i = i+1

allFeats = featMean
features = [featVar, featStd, featMin, featMax, featSkew, featKurtosis, featMeanCrossing, featMeanSpectralEnergy, featFourBin]
for feature in features:
    allFeats = np.append(allFeats, feature, axis=1)

print(np.shape(allFeats)) #4 samples by (12 channels by 13 features) = 4 x 156
        

(4, 156)


## Train classifiers 

In [123]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [124]:
#Random Forest
#Create and train classifier
y = np.reshape([1,2,3,4],[4,1])
#Train classifier. 
clfRf = RandomForestClassifier(n_jobs=2,random_state=0) #n_jobs to run in parallel. 
#Random_state is the seed used for the random number generator. 
clfRf.fit(allFeats,y)

C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [126]:
#Support Vector Classifier
clfSvc = svm.SVC(kernel='linear')
clfSvc.fit(allFeats,y)


C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Make Testing Data

In [127]:
indexStart = 700
indexStop = 1400

data = pd.read_csv('Elbowing.txt', sep="\t", header=None)
#data.columns = ["R_BI", "R_TRI", "L_BI", "L_TRI", "R_THI", "R_HAM", "L_THI", "L_HAM"]
elbow = data[indexStart:indexStop]

data = pd.read_csv('Frontkicking.txt', sep="\t", header=None)
frontKick = data[indexStart:indexStop]

data = pd.read_csv('Hamering.txt', sep="\t", header=None)
hamering = data[indexStart:indexStop]

data = pd.read_csv('Headering.txt', sep="\t", header=None)
headering = data[indexStart:indexStop]

frames = [elbow, frontKick, hamering, headering]

In [128]:
def appendEnergyVector(dataFrame):
    #take in the dataFrame, set new columns as a^2 + b^2 for each pair of 2. 
    energyCols = [8, 9, 10, 11]
    i = 0
    for col in energyCols:
        dataFrame[col] = np.square(dataFrame[i])+np.square(dataFrame[i+1])
        i = i+2
    return dataFrame

elbow = appendEnergyVector(elbow)
frontKick = appendEnergyVector(frontKick)
hamering = appendEnergyVector(hamering)
headering = appendEnergyVector(headering)

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [129]:
#Build feature vector for testing data


In [130]:
featMean = np.empty([4,12])
featVar = np.empty([4,12])
featStd = np.empty([4,12])
featMin = np.empty([4,12])
featMax = np.empty([4,12])
featSkew = np.empty([4,12])
featKurtosis = np.empty([4,12])
featMeanCrossing = np.empty([4,12])
featMeanSpectralEnergy = np.empty([4,12], dtype=complex)
featFourBin = np.empty([4,48])

i=0
for frame in frames:
    featMean[i][:] = np.mean(frame).values.reshape([1,12])
    featVar[i][:] = np.var(frame).values.reshape([1,12])
    featStd[i][:] = np.std(frame).values.reshape([1,12])
    featMin[i][:] = np.min(frame).values.reshape([1,12])
    featMax[i][:] = np.max(frame).values.reshape([1,12])
    featSkew[i][:] = skew(frame)
    featKurtosis[i][:] = kurtosis(frame)
    featMeanCrossing[i][:] = meanCrossing(frame)
    featMeanSpectralEnergy[i][:] = np.mean(np.square(np.fft.fft(frame)),0)
    featFourBin[i][:] = fourBin(frame)
    i = i+1

allFeats = featMean
features = [featVar, featStd, featMin, featMax, featSkew, featKurtosis, featMeanCrossing, featMeanSpectralEnergy, featFourBin]
for feature in features:
    allFeats = np.append(allFeats, feature, axis=1)

print(np.shape(allFeats)) #4 samples by (12 channels by 13 features) = 4 x 156

(4, 156)


## Test Classifier

### Random Forest

In [131]:
clfRf.predict(allFeats)

C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


array([1, 2, 3, 4])

In [152]:
clfRf.predict_proba(allFeats)

C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


array([[ 0.3,  0.3,  0.3,  0.1],
       [ 0.3,  0.4,  0. ,  0.3],
       [ 0.2,  0.2,  0.3,  0.3],
       [ 0.1,  0.4,  0. ,  0.5]])

In [154]:
y_test = [[1],[2],[3],[4]]
clfRf.score(allFeats, y_test)

C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


1.0

In [155]:
featureDf = pd.DataFrame(allFeats)
featureDf[0:155].head()
importances = list(zip(featureDf[0:155], clf.feature_importances_))
importances.sort(key=lambda x: x[1], reverse=True)
importances[0:20]

[(107, 0.16),
 (12, 0.10000000000000001),
 (75, 0.10000000000000001),
 (76, 0.10000000000000001),
 (4, 0.059999999999999998),
 (67, 0.059999999999999998),
 (130, 0.059999999999999998),
 (153, 0.059999999999999998),
 (2, 0.040000000000000001),
 (45, 0.040000000000000001),
 (85, 0.040000000000000001),
 (87, 0.040000000000000001),
 (104, 0.040000000000000001),
 (27, 0.033333333333333333),
 (122, 0.033333333333333333),
 (151, 0.033333333333333333),
 (0, 0.0),
 (1, 0.0),
 (3, 0.0),
 (5, 0.0)]

### Support Vector Classifier

In [149]:
preds = clfSvc.predict(allFeats)
preds

C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


array([1, 2, 1, 4])

In [150]:
y_test = [[1],[2],[3],[4]]
clfSvc.score(allFeats, y_test)

C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


0.75

# Depreciated

In [ ]:
#np.var(elbow)

In [ ]:
#np.std(elbow)

In [ ]:
#np.min(elbow)

In [ ]:
#np.max(elbow)

In [ ]:
#skew(elbow)

In [ ]:
#kurtosis(elbow)

In [ ]:
#Mean crossings
#centeredElbow = elbow - means
#((centeredElbow[0][:-1].values * centeredElbow[0][1:].values) < 0).sum()


In [ ]:
#Mean spectral energy
#fourier = np.fft.fft(elbow)
#uSpectralEnergy = np.mean(np.square(fourier),0)
#print(uSpectralEnergy)

In [ ]:
#Test classifier by passing in 700 new values from something, getting the features, then predicing. 
